# R-NaD Pokemon TCG Pocket Training on Colab

This notebook sets up the environment and runs the training script for R-NaD on Google Colab.

In [ ]:
# ==========================================
# settings
# ==========================================
# 1. Tailscale Auth Key
TAILSCALE_AUTH_KEY = "tskey-auth-..." 

# 2. ~/.ssh/id_rsa.pub
SSH_PUBLIC_KEY = "ssh-rsa AAAA..."

# ==========================================
# setup
# ==========================================
import os

def setup_tailscale_ssh():
    print("--- Tailscale & SSH Setup Started ---")

    print("[1/4] Installing Tailscale...")
    !curl -fsSL https://tailscale.com/install.sh | sh > /dev/null

    print("[2/4] Starting Tailscale...")
    get_ipython().system_raw('tailscaled --tun=userspace-networking --socks5-server=localhost:1055 &')
    !tailscale up --authkey=$TAILSCALE_AUTH_KEY --hostname=colab-tpu --ssh

    print("[3/4] Configuring SSH Server...")
    !apt-get install -y openssh-server > /dev/null
    !mkdir -p /var/run/sshd
    
    ssh_dir = os.path.expanduser("~/.ssh")
    if not os.path.exists(ssh_dir):
        os.makedirs(ssh_dir)
    
    with open(os.path.join(ssh_dir, "authorized_keys"), "w") as f:
        f.write(SSH_PUBLIC_KEY)
    
    # force pubkey
    !sed -i 's/#PasswordAuthentication yes/PasswordAuthentication no/' /etc/ssh/sshd_config
    !sed -i 's/#PubkeyAuthentication yes/PubkeyAuthentication yes/' /etc/ssh/sshd_config
    !service ssh start

    print("\n[4/4] Setup Complete!")
    print("--- Connection Info ---")
    !tailscale ip -4
    print(f"Command: ssh root@colab-tpu")
    print("------------------------")

setup_tailscale_ssh()